In [60]:
# riss.kr 에서 특정 키워드로 논문 / 학술 자료 검색하기

#Step 1. 필요한 모듈을 로딩합니다
from selenium import webdriver
import time 

#Step 2. 사용자에게 검색 관련 정보들을 입력 받습니다.
print("=" *100)
print(" 이 크롤러는 RISS 사이트의 논문 및 학술자료 수집용 웹크롤러입니다.")
print("=" *100)
query_txt = input('1.수집할 자료의 키워드는 무엇입니까?(여러개일 경우 , 로 구분하여 입력): ')
print(" 2. 위 키워드로 아래의 장르 중 어떤 장르의 정보를 수집할까요?")
print("1.학위논문", " "*8, "2.국내학술논문", " "*6, "3.해외학술논문", " "*6 ,"4.학술지")
print("5.단행본", " "*10, "6.공개강의", " "*10, "7.연구보고서")
info = {1:"학위논문", 2:"국내학술논문", 3:"해외학술논문", 4:"학술지", 5:"단행본", 6:"공개강의", 7:"연구보고서"}
query_txt2 = input('위 장르 중 수집할 장르의 번호를 입력하세요: ')
query_txt3 = info[int(query_txt2)]

#Step 3. 수집된 데이터를 저장할 파일 이름 입력받기 
fc_name = input('3.결과를 저장할 csv형식의 파일명을 쓰세요(예: c:\\temp\\riss.csv): ')
fx_name = input('4.결과를 저장할 xls형식의 파일명을 쓰세요(예: c:\\temp\\riss.xls): ')

#Step 4. 크롬 드라이버 설정 및 웹 페이지 열기
chrome_path = "C:/data/chromedriver"
driver = webdriver.Chrome(chrome_path)

url = 'http://www.riss.kr/'
driver.get(url)
time.sleep(2)

#Step 5. 자동으로 검색어 입력 후 조회하기
element = driver.find_element_by_id("query")
driver.find_element_by_id("query").click( )
element.send_keys(query_txt)
element.send_keys("\n")
time.sleep(2)

#Step 6.학위 논문 선택하기
driver.find_element_by_link_text(query_txt3).click()
time.sleep(2)

#Step 7.Beautiful Soup 로 본문 내용만 추출하기
from bs4 import BeautifulSoup
html_1 = driver.page_source
soup_1 = BeautifulSoup(html_1, 'html.parser')

content_1 = soup_1.find('div','srchResultListW').find_all('li')
for i in content_1 :
    print(i.get_text().replace("\n",""))

#Step 8. 총 검색 건수를 보여주고 수집할 건수 입력받기
import math
total_cnt = soup_1.find('div','searchBox').find('span','num').get_text()
print('검색하신 키워드 %s (으)로 총 %s 건의 학위논문이 검색되었습니다' %(query_txt,total_cnt))
collect_cnt = int(input('이 중에서 몇 건을 수집하시겠습니까?: '))
collect_page_cnt = math.ceil(collect_cnt / 10)
print('%s 건의 데이터를 수집하기 위해 %s 페이지의 게시물을 조회합니다.' %(collect_cnt,collect_page_cnt))

#Step 9. 각 항목별로 데이터를 추출하여 리스트에 저장하기
no2 = [ ]        #번호 저장
title2 = [ ]     #논문제목 저장
writer2 = [ ]    #논문저자 저장
org2 = [ ]       #소속기관 저장
no = 1
a1 = [ ] # 발표년도
b1 = [ ] # 논문집/자료집
c1 = [ ] # 논문URL정보

# 다음 페이지 번호 만들기
page_no=[ ]

for i in range(10,collect_cnt) :
    if i % 10 == 0 :
            page_no.append(i + 1)

for a in range(1, collect_page_cnt + 1) :
    
    html_2 = driver.page_source
    soup_2 = BeautifulSoup(html_2, 'html.parser')

    content_2 = soup_2.find('div','srchResultListW').find_all('li')
    
    for b in content_2 :    
        #1. 논문제목 있을 경우만
        try :
            title = b.find('div','cont').find('p','title').get_text()
        except :
            continue
        else :
            
            print('1.번호:',no)
            no2.append(no)
            

            print('2.논문제목:',title)
            title2.append(title)
            
            
            writer = b.find('span','writer').get_text()
            print('3.저자:',writer)
            writer2.append(writer)
            

            org = b.find('span','assigned').get_text()
            print('4.소속기관:' , org)
            org2.append(org)
            

            a2 = b.select_one('div.cont > p.etc > span:nth-child(3)').get_text()
            print('5.발표년도:' , a2)
            a1.append(a2)
            
            b2 = b.find('a').get_text()
            print('6.논문집/자료집:' , b2)
            b1.append(b2)


            c2 = b.find('a')["href"]
            print('7.논문 URL정보:' , c2)
            c1.append(c2)
            
            
            
            
            no += 1
            print("\n")
            
            if no > collect_cnt :
                break

            time.sleep(1)        # 페이지 변경 전 1초 대기 

        c = math.floor(a/10)-1 
        a += 1
        
        try:
            if a == page_no[c]:
                driver.find_element_by_link_text('다음 페이지로').click()
            else :
                driver.find_element_by_link_text('%s' %a).click() # 다음 페이지번호 클릭

        except:
                print("요청하신 작업이 모두 완료되었습니다")

# Step 10. 수집된 데이터를 xls와 csv 형태로 저장하기
import pandas as pd 

df = pd.DataFrame()
df['번호']=no2
df['제목']=pd.Series(title2)
df['작성자']=pd.Series(writer2)
df['소속기관']=pd.Series(org2)
df['발표년도']=pd.Series(a1)
df['논문집/자료집']=pd.Series(b1)
df['논문 URL정보']=pd.Series(c1)


# xls 형태로 저장하기
df.to_excel(fx_name,index=False, encoding="utf-8")

# csv 형태로 저장하기
df.to_csv(fc_name,index=False, encoding="utf-8-sig")

print('요청하신 데이터 수집 작업이 정상적으로 완료되었습니다')


 이 크롤러는 RISS 사이트의 논문 및 학술자료 수집용 웹크롤러입니다.
1.수집할 자료의 키워드는 무엇입니까?(여러개일 경우 , 로 구분하여 입력): 8
 2. 위 키워드로 아래의 장르 중 어떤 장르의 정보를 수집할까요?
1.학위논문          2.국내학술논문        3.해외학술논문        4.학술지
5.단행본            6.공개강의            7.연구보고서


KeyboardInterrupt: Interrupted by user

In [61]:
# riss.kr 에서 특정 키워드로 논문 / 학술 자료 검색하기

#Step 1. 필요한 모듈을 로딩합니다
from selenium import webdriver
import time 

#Step 2. 사용자에게 검색 관련 정보들을 입력 받습니다.
print("=" *100)
print(" 연습문제 : 저장할 내용을 목록으로 만들어서 xls, csv 형식으로 저장하기 ")
print("=" *100)
query_txt = input('1.크롤링할 키워드는 무엇입니까?: ')
collect_cnt = int(input('2.수집할 데이터는 몇 건입니까??: '))
import math
page_cnt = math.ceil(collect_cnt / 30)

#Step 3. 수집된 데이터를 저장할 파일 이름 입력받기 
fc_name = input('3.결과를 저장할 csv형식의 파일명을 쓰세요(예: c:\\temp\\riss.csv): ')
fx_name = input('4.결과를 저장할 xls형식의 파일명을 쓰세요(예: c:\\temp\\riss.xls): ')


#Step 4. 크롬 드라이버 설정 및 웹 페이지 열기
chrome_path = "C:/data/chromedriver"
driver = webdriver.Chrome(chrome_path)

url = 'http://naver.com'
driver.get(url)
time.sleep(2)

#Step 5. 자동으로 검색어 입력 후 조회하기
element = driver.find_element_by_id("query")
driver.find_element_by_id("query").click( )
element.send_keys(query_txt)
element.send_keys("\n")
time.sleep(2)

driver.find_element_by_link_text("VIEW").click()
time.sleep(2)

driver.find_element_by_link_text("블로그").click()
time.sleep(2)


#Step 5. 현재 페이지의 내용을 저장 목록을 만든 후 목록에 있는 내용을 파일에 저장하기
no = 1

# 자동 스크롤다운 함수
def scroll_down(driver):
  driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
  time.sleep(5)

i = 1
while (i <= page_cnt):
      scroll_down(driver) 
      i += 1
    

print("\n")

#Step 7.Beautiful Soup 로 본문 내용만 추출하기
from bs4 import BeautifulSoup
html_1 = driver.page_source
soup_1 = BeautifulSoup(html_1, 'html.parser')

content_1 = soup_1.find('ul','lst_total').find_all('li')
for i in content_1 :
    print(i.get_text().replace("\n",""))


# Step 9. 저장 목록을 만든 후 목록에 있는 내용을 파일에 저장하기

no2 = [ ]           # 게시글 번호 컬럼
title2 = [ ]        # 게시물 제목 컬럼
org2 = [ ]     # 게시글 내용 컬럼
bdate2 = [ ]        # 작성 일자 컬럼
nick2 = [ ]         # 블로그 닉네임


html_2 = driver.page_source
soup_2 = BeautifulSoup(html_2, 'html.parser')

content_2 = soup_2.find('ul','lst_total').find_all('li')
    
for b in content_2 :    

    print('1.번호:',no)
    no2.append(no)
            
    title = b.find('a','api_txt_lines total_tit').get_text()
    print('2.제목:',title)
    title2.append(title)
        
    try:
        org = b.find('div', 'api_txt_lines dsc_txt').get_text()
    except:
        org = '본문 요약 내용이 없습니다'
        org2.append(org)
        print('3.내용:',org)
    else:
        org2.append(org)
        print('3.내용:',org)
        
    bdate = b.find('span', 'sub_time sub_txt').get_text()
    print('4.작성일자:' , bdate)
    bdate2.append(bdate)

    nick = b.find('span','elss etc_dsc_inner').get_text()
    print('5.닉네임:',nick)
    nick2.append(nick)           
            
            

        
    if no == collect_cnt:
        break
        
    no += 1
    print("\n")
        
# Step 10. 수집된 데이터를 xls와 csv 형태로 저장하기
import pandas as pd 

df = pd.DataFrame()
df['번호']=no2
df['제목']=pd.Series(title2)
df['내용']=pd.Series(org2)
df['작성일자']=pd.Series(bdate2)
df['닉네임']=pd.Series(nick2)

# xls 형태로 저장하기
df.to_excel(fx_name,index=False, encoding="utf-8")

# csv 형태로 저장하기
df.to_csv(fc_name,index=False, encoding="utf-8-sig")

print("\n")
print("="*100)
print('요청하신 데이터 수집 작업이 정상적으로 완료되었습니다')


 연습문제 : 저장할 내용을 목록으로 만들어서 xls, csv 형식으로 저장하기 


KeyboardInterrupt: Interrupted by user

In [9]:
# riss.kr 에서 특정 키워드로 논문 / 학술 자료 검색하기

#Step 1. 필요한 모듈을 로딩합니다
from selenium import webdriver
import time 

#Step 2. 사용자에게 검색 관련 정보들을 입력 받습니다.
print("=" *100)
print(" 이 크롤러는 RISS 사이트의 논문 및 학술자료 수집용 웹크롤러입니다.")
print("=" *100)
query_txt = input('1.수집할 자료의 키워드는 무엇입니까?(여러개일 경우 , 로 구분하여 입력): ')

#Step 3. 수집된 데이터를 저장할 파일 이름 입력받기 
f_dir = input("2.파일을 저장할 폴더명만 쓰세요(기본값:c:\\temp\\):")
if f_dir == '' :
    f_dir="c:\\temp\\"

#Step 4. 크롬 드라이버 설정 및 웹 페이지 열기
chrome_path = "c:/data/chromedriver.exe"
driver = webdriver.Chrome(chrome_path)

url = 'http://www.riss.kr/'
driver.get(url)
time.sleep(2)

#Step 5. 자동으로 검색어 입력 후 조회하기
element = driver.find_element_by_id("query")
driver.find_element_by_id("query").click( )
element.send_keys(query_txt)
element.send_keys("\n")
time.sleep(2)

#Step 6.학위 논문 선택하기
driver.find_element_by_link_text('학위논문').click()
time.sleep(2)

#Step 7.Beautiful Soup 로 본문 내용만 추출하기
from bs4 import BeautifulSoup
html_1 = driver.page_source
soup_1 = BeautifulSoup(html_1, 'html.parser')

content_1 = soup_1.find('div','srchResultListW').find_all('li')
for i in content_1 :
    print(i.get_text().replace("\n",""))

#Step 8. 총 검색 건수를 보여주고 수집할 건수 입력받기
import math
total_cnt = soup_1.find('div','searchBox pd').find('span','num').get_text()
print('검색하신 키워드 %s (으)로 총 %s 건의 학위논문이 검색되었습니다' %(query_txt,total_cnt))
collect_cnt = int(input('이 중에서 몇 건을 수집하시겠습니까?: '))
collect_page_cnt = math.ceil(collect_cnt / 10)
print('%s 건의 데이터를 수집하기 위해 %s 페이지의 게시물을 조회합니다.' %(collect_cnt,collect_page_cnt))
print("\n")

#Step 9. 데이터 수집하기
no2=[]           # 게시글 번호 컬럼
title2=[ ]       # 게시글 제목 컬럼
author2=[]       # 논문 저자 컬럼
company2=[ ]     # 소속 기관 컬럼
date2=[ ]        # 게시글 날짜 컬럼
suksa2=[ ]       # 국내석사 컬럼
contents2=[]     # 초록내용
full_url2=[]     # 논문 원본 URL

no = 1           # 게시글 번호 초기값

# 다음 페이지 번호 만들기
page_no=[]

for i in range(10,collect_cnt) :
    if i % 10 == 0 :
            page_no.append(i + 1)
            
for a in range(1,collect_page_cnt+1) :
    print("\n")
    print("%s 페이지 내용 수집 시작합니다 =======================" %a)

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    content_list = soup.find('div','srchResultListW').find_all('li')

    for i in content_list:

        # 게시글 제목 체크하기
        try:
            title=i.find('div','cont').find('p','title').get_text().strip()
        except :
            continue 
        else :
            # 1.게시글 번호
            print("\n")
            print("%s 번째 정보를 추출하고 있습니다============" %no)
            no2.append(no)
            print("1.번호 : %s" %no)
            
            # 2. 논문 제목
            title2.append(title.strip())
            print("2.제목 : %s" %title.strip())

            # 3. 작성자
            try :
                author=i.find('div','cont').find('p','etc').find('span','writer').get_text().strip()
            except :
                author = '작성자가 없습니다'
                print("3.작성자 : %s" %author.strip())
                author2.append(author.strip())
            else :
                author2.append(author.strip())
                print("3.작성자 : %s" %author.strip())

            # 4. 소속기관
            try :
                company=i.find('div','cont').find('p','etc').find('span','assigned').get_text().strip()
            except :
                company='소속 기관이 없습니다'
                company2.append(company.strip())
                print("4.소속기관 : %s" %company.strip())
            else :
                company2.append(company.strip())
                print("4.소속기관 : %s" %company.strip())

            # 5. 발표날짜
            try :
                date_1 =i.find('div','cont').find('p','etc').find_all('span')
                date_2 = date_1[2].get_text().strip()
            except :
                date_2='발표날짜가 없습니다'
                date2.append(date_2)
                print("5.발표년도 : %s" %date_2)
            else :
                date2.append(date_2)
                print("5.발표년도 : %s" %date_2)

            # 6.학위여부
            try :
                suksa_1 =i.find('div','cont').find('p','etc').find_all('span')
                suksa_2 = suksa_1[3].get_text().strip()
            except :
                suksa_2='학위가 없습니다'
                suksa2.append(suksa_2)
                print("6.학위여부 : %s" %suksa_2)
            else :
                suksa2.append(suksa_2)
                print("6.학위여부 : %s" %suksa_2)

            # 7.초록 내용        
            url_1 = i.find('div','cont').find('p','title').find('a')['href']
            full_url = 'http://www.riss.kr'+url_1

            time.sleep(1)

            driver.get(full_url)

            html_1 = driver.page_source
            soup_1 = BeautifulSoup(html_1, 'html.parser')  
            try :
                cont=soup_1.find("div","text").find('p').get_text().replace("\n","").strip()
            except :
                cont='초록이 없습니다'
                contents2.append(cont)
                print("7.초록내용 : %s" %cont)
            else :
                contents2.append(cont)
                print("7.초록내용 : %s" %cont)

            time.sleep(1)

            # 8.논문 url 주소
            full_url2.append(full_url)
            print('8.논문 URL 주소:' , full_url)

            driver.back()  # 이전 페이지로 돌아가기

            time.sleep(2)

            no += 1
                            
    c = math.floor(a/10)-1 
    a += 1
    
    if a == page_no[c]:
        driver.find_element_by_link_text('다음 페이지로').click()
    else :
        driver.find_element_by_link_text('%s' %a).click() # 다음 페이지번호 클릭
        
print("요청하신 작업이 모두 완료되었습니다")

# Step 10. 수집된 데이터를 xls와 csv 형태로 저장하기
# 현재 날짜와 시간으로 폴더 만들고 파일 이름 설정하기
import time
import os

n = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' %(n.tm_year, n.tm_mon, n.tm_mday, n.tm_hour, n.tm_min, n.tm_sec)

os.makedirs(f_dir+'RISS'+'-'+s+'-'+'학위논문')

fc_name = f_dir+'RISS'+'-'+s+'-'+'학위논문'+'\\'+'RISS'+'-'+s+'-'+'학위논문'+'.csv'
fx_name = f_dir+'RISS'+'-'+s+'-'+'학위논문'+'\\'+'RISS'+'-'+s+'-'+'학위논문'+'.xls'

# 데이터 프레임 생성 후 xls , csv 형식으로 저장하기
import pandas as pd 

df = pd.DataFrame()
df['번호']=no2
df['제목']=pd.Series(title2)
df['저자']=pd.Series(author2)
df['소속(발행)기관']=pd.Series(company2)
df['날짜']=pd.Series(date2)
df['학위(논문일경우)']=pd.Series(suksa2)
df['초록(논문일경우)']=pd.Series(contents2)
df['자료URL주소']=pd.Series(full_url2)

# xls 형태로 저장하기
df.to_excel(fx_name,index=False, encoding="utf-8")

# csv 형태로 저장하기
df.to_csv(fc_name,index=False, encoding="utf-8-sig")

print('요청하신 데이터 수집 작업이 정상적으로 완료되었습니다')

 이 크롤러는 RISS 사이트의 논문 및 학술자료 수집용 웹크롤러입니다.
1.수집할 자료의 키워드는 무엇입니까?(여러개일 경우 , 로 구분하여 입력): 해양자원
2.파일을 저장할 폴더명만 쓰세요(기본값:c:\temp\):C:/data/
1지속가능한 해양관광 발전방안에 대한 연구 : 관광자원개발을 중심으로김선종연세대학교 행정대학원2008국내석사RANK : 13634431원문보기음성듣기삼면이 바다로 둘러싸여 천혜의 해양국가인 우리나라는 세계적 수준의 자연관광자원을 보유하고 있으며 연안을 중심으로 관광활동에 좋은 지형적 지리적 조건을 갖추고 있다. 특히, 우리나라 해안은 지형 및 지세, 기상 등 자원특성이 서로 달라 각 해안별로 독특한 자연조건을 갖고 있어 해양관광발전 잠재력이 매우 크다. 수질이 깨끗하고 백사장이 잘 발달된 동해안은 해수욕에 유리하고, 수심이 낮고 갯벌이 발달된 서해안은 생태관광의 보고이며, 기후가 온난하고 호수와 같은 잔잔한 내해를 이루고 있는 남해안은 스포츠형관광의 최적의 자연자원을 보유하고 있다.바다와 육지가 맞닿아 서로 밀접한 영향을 주고받으면서 특수한 자원?환경시스템을 이루는 연안(沿岸)은 해변?갯벌?만?삼각주 등 다양한 모습을 보이고 있으며, 그동안 수산물의 산란장과 서식지로서 항만?임해공단 등 경제활동공간으로서 도시가 발달하고 인구가 집중되어있는 중요한 활동공간이었으나, 이제는 레저스포츠 등 관광활동공간으로서 그 중요성이 더해가고 있다.우리나라는 우수한 자연관광자원을 보유하고 있으나 그동안 육상위주의 레저 및 관광정책추진으로 해양관광은 크게 발전하지 못하고 있다. 그러나 국민들의 소득수준 증가와 더불어 주2일 휴무제 정착에 따른 여가시간의 증대 등 라이프스타일 변화로 직접체험을 즐길 수 있는 해양레저스포츠 등 관광수요는 크게 증가할 것으로 보인다.이러한 좋은 여건에도 불구하고 자치단체에서는 사전 타당성 검토가 부족한 대단위 관광단지 위주의 개발계획들만을 경쟁적으로 추진하고 있어 증가하고 있는 해양관광수요에는 효과적으로 대응하지 못하는 

이 중에서 몇 건을 수집하시겠습니까?: 40
40 건의 데이터를 수집하기 위해 4 페이지의 게시물을 조회합니다.




1 페이지 내용 수집 시작합니다 =======================


1 번째 정보를 추출하고 있습니다============
1.번호 : 1
2.제목 : 지속가능한 해양관광 발전방안에 대한 연구 : 관광자원개발을 중심으로
3.작성자 : 김선종
4.소속기관 : 연세대학교 행정대학원
5.발표년도 : 2008
6.학위여부 : 국내석사
7.초록내용 : 삼면이 바다로 둘러싸여 천혜의 해양국가인 우리나라는 세계적 수준의 자연관광자원을 보유하고 있으며 연안을 중심으로 관광활동에 좋은 지형적 지리적 조건을 갖추고 있다. 특히, 우리나...
8.논문 URL 주소: http://www.riss.kr/search/detail/DetailView.do?p_mat_type=be54d9b8bc7cdb09&control_no=42ac5dcefee98705ffe0bdc3ef48d419


2 번째 정보를 추출하고 있습니다============
1.번호 : 2
2.제목 : 여수지역 해양생물자원을 활용한 관광지 활성화에 관한 연구
3.작성자 : 권진구
4.소속기관 : 전남대학교 대학원
5.발표년도 : 2021
6.학위여부 : 국내박사
7.초록내용 : 초록이 없습니다
8.논문 URL 주소: http://www.riss.kr/search/detail/DetailView.do?p_mat_type=be54d9b8bc7cdb09&control_no=3a475d91f91e55b2ffe0bdc3ef48d419


3 번째 정보를 추출하고 있습니다============
1.번호 : 3
2.제목 : 해양관광 자원 개발에 대한 연구 : 중국 칭다오와 부산을 중심으로
3.작성자 : 장잉잉
4.소속기관 : 경기대학교 대학원
5.발표년도 : 2019
6.학위여부 : 국내석사
7.초록내용 : 21세기는 해양 경제 시대이므로 해양산업은 세계 각국에서 중요시하고 있다. 특히 연해 국가들은 해양



20 번째 정보를 추출하고 있습니다============
1.번호 : 20
2.제목 : 海洋에 關한 國際立法의 方向 : 海産資源保存問題를 中心으로
3.작성자 : 강대경
4.소속기관 : 고려대학교
5.발표년도 : 1956
6.학위여부 : 국내석사
7.초록내용 : 초록이 없습니다
8.논문 URL 주소: http://www.riss.kr/search/detail/DetailView.do?p_mat_type=be54d9b8bc7cdb09&control_no=4fd0fe1249bb9e65ffe0bdc3ef48d419


3 페이지 내용 수집 시작합니다 =======================


21 번째 정보를 추출하고 있습니다============
1.번호 : 21
2.제목 : 한국 서남해안 완도해역의 유용 해양 무척추동물자원의 분포
3.작성자 : 정태원
4.소속기관 : 조선대학교 대학원
5.발표년도 : 2006
6.학위여부 : 국내석사
7.초록내용 : The present study was performed to elucidate the distribution of marine invertebrates useful for the materials of pharmacological natural products in Wando, south-western coast of Korea. The study was based on the field survey in 2006 with investigati...
8.논문 URL 주소: http://www.riss.kr/search/detail/DetailView.do?p_mat_type=be54d9b8bc7cdb09&control_no=67dd234d714c40bcffe0bdc3ef48d419


22 번째 정보를 추출하고 있습니다============
1.번호 : 22
2.제목 : 북태평양의 기후변화에 따른 우리나라 해역별 해양환경 및 수산생물 자원의 반응과 기작
3.작성자 : 정해근
4.소속기관 : 강릉원주대학교
5.

<ipython-input-9-42f793e6bf87>:215: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  df.to_excel(fx_name,index=False, encoding="utf-8")


In [68]:
# riss.kr 에서 특정 키워드로 논문 / 학술 자료 검색하기

#Step 1. 필요한 모듈을 로딩합니다
from selenium import webdriver
import time 

#Step 2. 사용자에게 검색 관련 정보들을 입력 받습니다.
print("=" *100)
print(" 연습문제 6-2: 청와대 국민청원 게시판 크롤링하기")
print("=" *100)
collect_cnt = int(input('1.크롤링할 건 수는 몇 건입니까?: '))

#Step 3. 수집된 데이터를 저장할 파일 이름 입력받기 
f_dir = input("2.결과 파일을 저장할 폴더명만 쓰세요(기본값:c:\\temp\\):")
if f_dir == '' :
    f_dir="c:\\temp\\"

#Step 4. 크롬 드라이버 설정 및 웹 페이지 열기
chrome_path = "c:/data/chromedriver.exe"
driver = webdriver.Chrome(chrome_path)

url = 'http://www1.president.go.kr/petitions'
driver.get(url)
time.sleep(2)


#Step 8. 총 검색 건수를 보여주고 수집할 건수 입력받기
import math

collect_page_cnt = math.ceil(collect_cnt / 7)
print('%s 건의 데이터를 수집하기 위해 %s 페이지의 게시물을 조회합니다.' %(collect_cnt,collect_page_cnt))
print("\n")

#Step 9. 데이터 수집하기
no2=[]           # 청원 번호 컬럼
title2=[ ]       # 청원 제목 컬럼
num2=[]       # 참여 인원 컬럼
cat2=[ ]     # 카테고리 컬럼
sdate2=[ ]        # 청원시작일 컬럼
fdate2=[ ]       # 청원종료일 컬럼
contents2=[]     # 청원내용


# 다음 페이지 번호 만들기
page_no=[]

for i in range(10,collect_cnt) :
    if i % 7 == 0 :
            page_no.append(i + 1)
            
for a in range(1,collect_page_cnt+1) :
    print("\n")
    print("%s 페이지 내용 수집 시작합니다 =======================" %a)

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    content_list = soup.find('ul','petition_list').find_all('li')

    for i in content_list:

        # 게시글 제목 체크하기
        try :
            title=i.find('a','cb relpy_w')
        except :
            continue 
        else :
            # 1. 청원 번호
            no = i.find('div', 'bl_no').get_text().replace("번호", "").strip()
            print("\n")
            print("%s 번째 정보를 추출하고 있습니다============" %no)
            no2.append(no)
            print("1.청원번호 : %s" %no)
            
            # 2. 청원 제목
            title = title.get_text().replace("제목", "").strip()
            title2.append(title)
            print("2.청원제목 : %s" %title.strip())
            
            # 3. 참여인원
            num = i.find('div', 'bl_agree cs').get_text().replace("참여인원", "").strip()
            num2.append(num)
            print("3.참여인원 : %s"%num)
            
            url_1 = i.find('div', 'bl_subject').find('a')['href']
            full_url = 'http://www1.president.go.kr/'+url_1

            time.sleep(1)

            driver.get(full_url)

            html_1 = driver.page_source
            soup_1 = BeautifulSoup(html_1, 'html.parser')  
            
            list = [cat2, sdate2, fdate2]
            
            # 4. 카테고리
            cat = soup_1.select('cont_view > div.cs_area > div.new_contents > div > div.petitionsView_left > div > div.petitionsView_info > ul > li:nth-child(1)')
            sdate = soup_1.select('cont_view > div.cs_area > div.new_contents > div > div.petitionsView_left > div > div.petitionsView_info > ul > li:nth-child(2)')
            fdate = soup_1.select('cont_view > div.cs_area > div.new_contents > div > div.petitionsView_left > div > div.petitionsView_info > ul > li:nth-child(3)')
                    
            
            try :
                cat= cat.replace("카테고리","").strip()
            except :
                cat='카테고리가 없습니다'
                cat2.append(cat)
                print("4.카테고리 : %s" %cat)
            else :
                cat2.append(cat)
                print("4.카테고리 : %s" %cat)
            
            # 5. 청원시작일
            try :
                sdate= sdate.replace("청원시작","").strip()
            except :
                sdate='청원시작이 없습니다'
                sdate2.append(sdate)
                print("5.청원시작 : %s" %sdate)
            else :
                sdate2.append(sdate)
                print("5.청원시작 : %s" %sdate)

            # 6. 청원종료일
            try :
                fdate=fdate.replace("청원마감","").strip()
            except :
                fdate='청원마감이 없습니다'
                fdate2.append(fdate)
                print("6.청원마감 : %s" %fdate)
            else :
                fdate2.append(fdate)
                print("6.청원마감 : %s" %fdate)        
            
            
            # 7. 청원내용
            try :
                contents =soup_1.find("div","View_write").get_text().replace("\n","").strip()
            except :
                contents='청원내용이 없습니다.'
                contents2.append(contents)
                print("7.청원내용 : %s" %contents)
            else :
                contents2.append(contents)
                print("7.청원내용 : %s" %contents)

            driver.back()  # 이전 페이지로 돌아가기

            time.sleep(2)

                            
    c = math.floor(a/7)-1 
    a += 1
    
    try:
        driver.find_element_by_link_text('%s' &a).click()
    except:
        driver.find_element_by_link_text('Next').click()
    
    '''
    if a == page_no[c]:
        driver.find_element_by_link_text('다음 페이지로').click()
    else :
        driver.find_element_by_link_text('%s' %a).click() # 다음 페이지번호 클릭
    '''    
print("요청하신 작업이 모두 완료되었습니다")

# Step 10. 수집된 데이터를 xls와 csv 형태로 저장하기
# 현재 날짜와 시간으로 폴더 만들고 파일 이름 설정하기
import time
import os

n = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' %(n.tm_year, n.tm_mon, n.tm_mday, n.tm_hour, n.tm_min, n.tm_sec)

os.makedirs(f_dir+'청와대'+'-'+s+'-'+'국민청원')

fc_name = f_dir+'청와대'+'-'+s+'-'+'국민청원'+'\\'+'RISS'+'-'+s+'-'+'국민청원'+'.csv'
fx_name = f_dir+'청와대'+'-'+s+'-'+'국민청원'+'\\'+'RISS'+'-'+s+'-'+'국민청원'+'.xls'

# 데이터 프레임 생성 후 xls , csv 형식으로 저장하기
import pandas as pd 

df = pd.DataFrame()
df['청원번호']=pd.Series(no2)
df['청원제목']=pd.Series(title2)
df['참여인원']=pd.Series(num2)
df['카테고리']=pd.Series(cat2)
df['청원시작일']=pd.Series(sdate2)
df['청원종료일']=pd.Series(fdate2)
df['청원내용']=pd.Series(contents2)

# xls 형태로 저장하기
df.to_excel(fx_name,index=False, encoding="utf-8")

# csv 형태로 저장하기
df.to_csv(fc_name,index=False, encoding="utf-8-sig")

print('요청하신 데이터 수집 작업이 정상적으로 완료되었습니다')

 연습문제 6-2: 청와대 국민청원 게시판 크롤링하기
1.크롤링할 건 수는 몇 건입니까?: 8
2.결과 파일을 저장할 폴더명만 쓰세요(기본값:c:\temp\):C:/data/
8 건의 데이터를 수집하기 위해 2 페이지의 게시물을 조회합니다.




1 페이지 내용 수집 시작합니다 =======================


734 번째 정보를 추출하고 있습니다============
1.청원번호 : 734
2.청원제목 : 전국의 아이를 원하는 난임부부들에게 힘을주세요.
3.참여인원 : 591명
4.카테고리 : 카테고리가 없습니다
5.청원시작 : 청원시작이 없습니다
6.청원마감 : 청원마감이 없습니다
7.청원내용 : 갈수록 저출산 문제가 심해지고 있습니다.자발적으로 아이를 갖지않는 사람들도 많지만,아이를 갖고싶어도 그렇지 못한 사람들도 많습니다.저는 결혼2년차로원인불명 난임입니다.벌써 냉동으로 시험관 5차까지 진행한상태이고 이제 지원은 끝낫고앞으로 자비로 해야합니다.한번 이식때마다 몇십~몇백은 들어갑니다.병원마다 같은주사지만 가격대도 천차만별입니다.이런 주사제, 경구약등등 시술에 필요한 약들의 가격을 병원마다 모두 일정하게 맞춰주세요. 맞벌이 부부지만 비용적인 문제때문에 아이갖는것을 포기할까 하는 생각이듭니다..저출산..저출산 하는데난임병원 가보면 사람들이 바글바글합니다.유명한 병원은 예약하고 대기만 2~3시간 걸립니다. 놀랐습니다. 이렇게 아이를 원하는 사람들이 많은데뭐가 저출산이라는거지?제 주변에도 난임부부들이 많습니다.이중에서는 비용문제로 아기를 포기한 사람들도 있습니다.정말 저출산의 문제를 해결하고 싶다면아이를 원하는 부부들에게 아낌없는 지원이 필요하다고 생각됩니다. 현재 난임부부를 위한 지원정책이 마련되어있지만이는 시술을 하기에 턱없이 부족합니다.한번 시술에 성공하면 더없이 좋겠지만대부분의 난임부부들은 수차례 시도를 합니다.시술에 필요한 거의 모든 처치, 약물들이 비급여이고그로인해 사람마다 처방이 다르겠지만,한,두번 병원에 다녀오면 지원금은 다 써집니다

KeyboardInterrupt: 

In [ ]:
 for i in list[cat2, sdate2, fdate2]:
        soup_1.select_one('> div > div.petitionsView_left > div > div.petitionsView_info > ul > li:nth-child({}) > p'.format(i))

In [ ]:
#cont_view > div.cs_area > div.new_contents > div > div.petitionsView_left > div > div.petitionsView_info > ul > li:nth-child(2) > p

In [46]:
name = 10

print('dfsf{}d'.format(name))

dfsf10d


In [ ]:
#cont_view > div.cs_area > div.new_contents > div > div.petitionsView_left > div > div.petitionsView_info > ul

In [54]:
print("::")

::
